#Install GROMACS and backup it on Google Drive

This notebook execute the following operations:
1. Compilation of the GROMCS software during a Google Colab session
2. Backing up the compiled software into your Google Drive

Using the backed-up software, you does not need to recompile it.

Inspired by IBM3202 notebooks: https://github.com/pb3lab/ibm3202

#**Before we start...**

##**1. Copy this notebook to your Drive, otherwise the installation will not work**

You can do this by clicking on the **Save a Copy to Drive** option on the **File** menu.

##**2. Watch out for idle timeouts on Google Colab**

To be able to offer free computational resources on the cloud, Colab has **idle timeouts** that disconnect the runtime session if left unattended for too long and **maximum lifetimes**. While the maximum lifetime on a standard Google Colab runtime can be as much as 12 hours, the **idle timeouts vary over time**. 

Thus, is is imperative that you keep an eye on your software installation every 10-15 min to avoid idle timeouts that would interrupt the whole process.

##**3. Where is the installed software located after backing up on Google Drive?**

All installed software will be located in a new **Gromacs** folder in your Google Drive account.

# Runtime setup and environment check



**VERY IMPORTANT FIRST STEP:** Go To *Runtime* -> *Change Runtime Type* and select GPU!

We will first check the Google Colab resources and CMake version.

In [ ]:
# Check the Google Colab resources and CMake version
# Must be: 1 GPU and 2 CPU with 12GB RAM and at least 30 GB available on root filesystem
%%bash
echo -e "# CPU Details"
lscpu
echo -e "\n# RAM Details"
free -h
echo -e "\n# NVIDIA Details"
nvidia-smi
echo -e "\n# CMake Details"
cmake --version
echo -e "\n# Filesystems Details"
df -h
echo -e "\n# Current Working Directory"
pwd

# Installation of latest CMake version



We now install the latest CMake version (as GROMACS 2021 require at least CMake 3.13).

In [ ]:
# Upgrade CMake at latest version via pip
!pip install cmake --upgrade

In [ ]:
# Check again CMake version
%%bash
echo "# CMake Details"
cmake --version

# Installation of GROMACS

## Downloading and installing GROMACS

We now select GROMACS version, download it onto Google Colab, and then install it. The installation takes **~40 min**.

In [ ]:
# Specify GROMACS version
COLAB_GROMACS_VERSION = "2021.5"

# -------- DOES NOT MODIFY AFTER THIS POINT !!!! --------

# Import needed general Python modules
import os

# Build version-related variables
COLAB_GROMACS_DOWNLOAD_LINK = "https://ftp.gromacs.org/gromacs/gromacs-%s.tar.gz" % COLAB_GROMACS_VERSION
COLAB_GROMACS_ARCHIVE_PATH = "/content/gromacs_src/gromacs-%s.tar.gz" % COLAB_GROMACS_VERSION
COLAB_GROMACS_SOURCE_PATH = "/content/gromacs_src/gromacs-%s" % COLAB_GROMACS_VERSION
COLAB_GROMACS_BUILD_PATH = "/content/gromacs_src/gromacs-%s/build" % COLAB_GROMACS_VERSION
COLAB_GROMACS_INSTALL_PATH = "/content/gromacs-%s" % COLAB_GROMACS_VERSION
COLAB_GROMACS_ENV_PATH = "/content/gromacs-%s/bin/GMXRC" % COLAB_GROMACS_VERSION
COLAB_GROMACS_GDRIVE_FS = "/content/gdrive"
COLAB_GROMACS_GDRIVE_DIR = "/content/gdrive/My Drive/GROMACS/"
COLAB_GROMACS_GDRIVE_ARCHIVE = "/content/gdrive/My Drive/GROMACS/gromacs-%s.tar.gz" % COLAB_GROMACS_VERSION

# Print GROMACS version selected
print("# GROMACS version selected : {COLAB_GROMACS_VERSION}")

# Print GROMACS custom variables
print("\n# GROMACS custom variables:")
print("- GROMACS download link  : %s" % COLAB_GROMACS_DOWNLOAD_LINK)
print("- GROMACS archive path   : %s" % COLAB_GROMACS_ARCHIVE_PATH)
print("- GROMACS source path    : %s" % COLAB_GROMACS_SOURCE_PATH)
print("- GROMACS build path     : %s" % COLAB_GROMACS_BUILD_PATH)
print("- GROMACS install path   : %s" % COLAB_GROMACS_INSTALL_PATH)
print("- GROMACS bin env path   : %s" % COLAB_GROMACS_ENV_PATH)
print("- GROMACS GDrive fs      : %s" % COLAB_GROMACS_GDRIVE_FS)
print("- GROMACS GDrive folder  : %s" % COLAB_GROMACS_GDRIVE_DIR)
print("- GROMACS GDrive archive : %s" % COLAB_GROMACS_GDRIVE_ARCHIVE)

# Save variables needed to copile process in temp files
!echo {COLAB_GROMACS_VERSION} > /tmp/colab_gromacs_version
!echo {COLAB_GROMACS_BUILD_PATH} > /tmp/colab_gromacs_build_path
!echo {COLAB_GROMACS_INSTALL_PATH} > /tmp/colab_gromacs_install_path
!echo {COLAB_GROMACS_ENV_PATH} > /tmp/colab_gromacs_env_path

In [ ]:
# Downloading GROMACS
!mkdir -p $(dirname {COLAB_GROMACS_ARCHIVE_PATH})
!wget -O {COLAB_GROMACS_ARCHIVE_PATH} {COLAB_GROMACS_DOWNLOAD_LINK}

In [ ]:
# Extract the downloaded folder containing GROMACS
!rm -rf ${COLAB_GROMACS_SOURCE_PATH}
os.makedirs(COLAB_GROMACS_SOURCE_PATH, exist_ok=True)
!tar xfz {COLAB_GROMACS_ARCHIVE_PATH} -C $(dirname {COLAB_GROMACS_SOURCE_PATH})
print("# GROMACS extraction completed")

In [ ]:
%%bash
# Load variables
export COLAB_GROMACS_VERSION=$(cat /tmp/colab_gromacs_version)
export COLAB_GROMACS_BUILD_PATH=$(cat /tmp/colab_gromacs_build_path)
export COLAB_GROMACS_INSTALL_PATH=$(cat /tmp/colab_gromacs_install_path)

# Create a "build" folder inside the extracted folder (with a cleanup before)
rm -rf ${COLAB_GROMACS_BUILD_PATH}
mkdir -p ${COLAB_GROMACS_BUILD_PATH}

# Setup our GROMACS installation options, including GPU and a user-defined folder
cd ${COLAB_GROMACS_BUILD_PATH}
if [[ ${COLAB_GROMACS_VERSION} = 2020.* ]]
then
  cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=on -DCMAKE_INSTALL_PREFIX=${COLAB_GROMACS_INSTALL_PATH}
  RETURNCODE=$?
elif [[ ${COLAB_GROMACS_VERSION} = 2021.* ]]
then
  cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=CUDA -DCMAKE_INSTALL_PREFIX=${COLAB_GROMACS_INSTALL_PATH}
  RETURNCODE=$?
else
  echo "Unsupported GROMACS version: ${COLAB_GROMACS_VERSION}"
  exit 1
fi
[[ ${RETURNCODE} != 0 ]] && echo "# GROMACS CMake failed" && exit 1
echo -e "# GROMACS CMake set up completed\n"

# Compile GROMACS and install it inside the user-defined folder
make -j 2
[[ $? != 0 ]] && echo -e "\n# GROMACS building failed\n" && exit 1
echo -e "\n# GROMACS building completed\n"
make -j 2 check
[[ $? != 0 ]] && echo -e "\n# GROMACS testing failed\n" && exit 1
echo -e "\n# GROMACS testing completed\n"
make install
[[ $? != 0 ]] && echo -e "\n# GROMACS installation failed\n" && exit 1
echo -e "\n# GROMACS installation completed. Please check if any errors occurred during installation\n"

In [ ]:
%%bash
cd /content/gromacs_src/gromacs-2021.5/build
make install

We now check that the installation was successful by loading the GROMACS PATH onto Google Colab.

In [ ]:
# Checking that GROMACS was successfully installed
%%bash
export COLAB_GROMACS_ENV_PATH=$(cat /tmp/colab_gromacs_env_path)
source ${COLAB_GROMACS_ENV_PATH}
gmx -h

##Backing up GROMACS on Google Drive

Once GROMACS is successfully compiled and installed on Google Colab, you can **save a back up** of GROMACS into your **Google Drive**. To do this, we first need to **Mount** your Google Drive into Google Colab to be able to transfer files, followed by backing up your GROMACS installation for future use, as follows:

In [ ]:
# First, we will mount your Google Drive to be able to transfer files
# You will be requested to authorize this Google Drive mounting
# By entering an authorization code into an input box and then pressing Enter
from google.colab import drive
drive.mount(COLAB_GROMACS_GDRIVE_FS)

print("\n# Google Drive Filesystem:")
!df -h {COLAB_GROMACS_GDRIVE_FS}

In [ ]:
# Then, we will ensure that the GROMACS folder exists in Google Drive
os.makedirs(COLAB_GROMACS_GDRIVE_DIR, exist_ok=True)
print("GROMACS folder present in Google Drive")

print("\n# Google Drive GROMACS folder content:")
!ls -l "{COLAB_GROMACS_GDRIVE_DIR}"

In [ ]:
# Lastly, we created directly in Google Drive GROMACS folder an archive of GROMACS software
!tar -C "{COLAB_GROMACS_INSTALL_PATH}/.." -zcvf "{COLAB_GROMACS_GDRIVE_ARCHIVE}" $(basename "{COLAB_GROMACS_INSTALL_PATH}")
print("\n# GROMACS backup completed!")

print("\n# Google Drive GROMACS folder content:")
!ls -l "{COLAB_GROMACS_GDRIVE_DIR}"